In [1]:
#Instalando o pacote
%pip install gurobipy

#Trabalho Python - Mestrado
#Disciplina Pesquisa Operacional 1

#Utilizarei o Gurobi, para resolver
import gurobipy as gp
import pandas as pd
from gurobipy import GRB

#conjuntos
estudantes = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
cursos = [1, 2, 3, 4, 5, 6]

#Definindo um modelo
m = gp.Model('problema_Mestrado')

Set parameter Username


In [2]:
#pontos por curso
path = '/Users/TullioPires/Python-Mestrado/dados.csv'
pontos = pd.read_csv(path, index_col=[0,1], squeeze=True)

#Pivotar a tabela, para ler
pontos.reset_index().pivot(index = 'Estudante', columns = 'Curso', values = 'Pontos')

/var/folders/f8/sx2sm28s5xn92g54_n5vypwh0000gn/T/ipykernel_49012/900302864.py:3: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  pontos = pd.read_csv(path, index_col=[0,1], squeeze=True)


Curso,1,2,3,4,5,6
Estudante,,,,,,
1,20,40,50,30,90,100
2,90,100,80,70,10,40
3,25,40,30,80,95,90
4,80,50,60,80,30,40
5,75,60,90,100,50,40
6,60,40,90,10,80,80
7,45,40,70,60,55,60
8,30,100,40,70,90,55
9,80,60,100,70,65,80


In [3]:
#Capacidade dos cursos
capacidade = pd.Series([6, 8, 5, 5, 6, 5], index = cursos, name = 'Capacidade')
capacidade.to_frame()

,Capacidade
1,6
2,8
3,5
4,5
5,6
6,5


In [4]:
#Necessidade de estudantes
nec_estudantes = pd.Series([2, 2, 2, 2, 2, 2, 2, 2, 2, 2], index = estudantes, name = 'Nec de estudantes')
nec_estudantes.to_frame()

,Nec de estudantes
1,2
2,2
3,2
4,2
5,2
6,2
7,2
8,2
9,2
10,2


In [5]:
#Criando as variáveis de decisão
m = gp.Model('problema_Mestrado')
x = m.addVars(estudantes, cursos, vtype = GRB.BINARY,name = 'estudante_curso')
m.update()
x

{(1, 1): <gurobi.Var estudante_curso[1,1]>,
 (1, 2): <gurobi.Var estudante_curso[1,2]>,
 (1, 3): <gurobi.Var estudante_curso[1,3]>,
 (1, 4): <gurobi.Var estudante_curso[1,4]>,
 (1, 5): <gurobi.Var estudante_curso[1,5]>,
 (1, 6): <gurobi.Var estudante_curso[1,6]>,
 (2, 1): <gurobi.Var estudante_curso[2,1]>,
 (2, 2): <gurobi.Var estudante_curso[2,2]>,
 (2, 3): <gurobi.Var estudante_curso[2,3]>,
 (2, 4): <gurobi.Var estudante_curso[2,4]>,
 (2, 5): <gurobi.Var estudante_curso[2,5]>,
 (2, 6): <gurobi.Var estudante_curso[2,6]>,
 (3, 1): <gurobi.Var estudante_curso[3,1]>,
 (3, 2): <gurobi.Var estudante_curso[3,2]>,
 (3, 3): <gurobi.Var estudante_curso[3,3]>,
 (3, 4): <gurobi.Var estudante_curso[3,4]>,
 (3, 5): <gurobi.Var estudante_curso[3,5]>,
 (3, 6): <gurobi.Var estudante_curso[3,6]>,
 (4, 1): <gurobi.Var estudante_curso[4,1]>,
 (4, 2): <gurobi.Var estudante_curso[4,2]>,
 (4, 3): <gurobi.Var estudante_curso[4,3]>,
 (4, 4): <gurobi.Var estudante_curso[4,4]>,
 (4, 5): <gurobi.Var estudante_c

In [6]:
#Criando as restrições
#Restrição de curso
restricao_curso = m.addConstrs((x.sum(i, '*') == 2 for i in estudantes), 
                               name = 'restrição de curso')
m.update()
restricao_curso

{1: <gurobi.Constr restrição de curso[1]>,
 2: <gurobi.Constr restrição de curso[2]>,
 3: <gurobi.Constr restrição de curso[3]>,
 4: <gurobi.Constr restrição de curso[4]>,
 5: <gurobi.Constr restrição de curso[5]>,
 6: <gurobi.Constr restrição de curso[6]>,
 7: <gurobi.Constr restrição de curso[7]>,
 8: <gurobi.Constr restrição de curso[8]>,
 9: <gurobi.Constr restrição de curso[9]>,
 10: <gurobi.Constr restrição de curso[10]>}

In [7]:
#Restrição de capacidade
restricao_capacidade = m.addConstrs((x.sum('*', j) <= capacidade[j] for j in cursos),
                                  name = 'restricao de capacidade')

m.update()
restricao_capacidade

{1: <gurobi.Constr restricao de capacidade[1]>,
 2: <gurobi.Constr restricao de capacidade[2]>,
 3: <gurobi.Constr restricao de capacidade[3]>,
 4: <gurobi.Constr restricao de capacidade[4]>,
 5: <gurobi.Constr restricao de capacidade[5]>,
 6: <gurobi.Constr restricao de capacidade[6]>}

In [8]:
#Criando a função objetivo
m.setObjective(gp.quicksum(pontos[i,j]*x[i,j] for i in estudantes for j in cursos), GRB.MAXIMIZE)

In [9]:
#Escrevendo o problema, pra checar se está correto
m.write('problema mestrado.lp')

In [10]:
#Resolvendo
m.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[rosetta2])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 16 rows, 60 columns and 120 nonzeros
Model fingerprint: 0xe2d80cd4
Variable types: 0 continuous, 60 integer (60 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 8e+00]
Found heuristic solution: objective 1300.0000000
Presolve time: 0.01s
Presolved: 16 rows, 60 columns, 120 nonzeros
Variable types: 0 continuous, 60 integer (60 binary)
Found heuristic solution: objective 1745.0000000

Root relaxation: cutoff, 30 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0      1745.00000 1745.00000  0.00%     -    0s

Explored 1 nodes

In [11]:
#Obtendo os valores
xvals = {k: v.x for k,v in x.items() if v.x > 0}
xvals

{(1, 5): 1.0,
 (1, 6): 1.0,
 (2, 1): 1.0,
 (2, 2): 1.0,
 (3, 5): 1.0,
 (3, 6): 1.0,
 (4, 1): 1.0,
 (4, 4): 1.0,
 (5, 3): 1.0,
 (5, 4): 1.0,
 (6, 3): 1.0,
 (6, 5): 1.0,
 (7, 3): 1.0,
 (7, 6): 1.0,
 (8, 2): 1.0,
 (8, 5): 1.0,
 (9, 1): 1.0,
 (9, 3): 1.0,
 (10, 3): 1.0,
 (10, 6): 1.0}

In [12]:
all_vars = {v.varName: v.x for v in m.getVars()}
all_vars

{'estudante_curso[1,1]': 0.0,
 'estudante_curso[1,2]': 0.0,
 'estudante_curso[1,3]': 0.0,
 'estudante_curso[1,4]': 0.0,
 'estudante_curso[1,5]': 1.0,
 'estudante_curso[1,6]': 1.0,
 'estudante_curso[2,1]': 1.0,
 'estudante_curso[2,2]': 1.0,
 'estudante_curso[2,3]': 0.0,
 'estudante_curso[2,4]': 0.0,
 'estudante_curso[2,5]': 0.0,
 'estudante_curso[2,6]': 0.0,
 'estudante_curso[3,1]': 0.0,
 'estudante_curso[3,2]': 0.0,
 'estudante_curso[3,3]': 0.0,
 'estudante_curso[3,4]': 0.0,
 'estudante_curso[3,5]': 1.0,
 'estudante_curso[3,6]': 1.0,
 'estudante_curso[4,1]': 1.0,
 'estudante_curso[4,2]': 0.0,
 'estudante_curso[4,3]': 0.0,
 'estudante_curso[4,4]': 1.0,
 'estudante_curso[4,5]': 0.0,
 'estudante_curso[4,6]': 0.0,
 'estudante_curso[5,1]': 0.0,
 'estudante_curso[5,2]': 0.0,
 'estudante_curso[5,3]': 1.0,
 'estudante_curso[5,4]': 1.0,
 'estudante_curso[5,5]': 0.0,
 'estudante_curso[5,6]': 0.0,
 'estudante_curso[6,1]': 0.0,
 'estudante_curso[6,2]': 0.0,
 'estudante_curso[6,3]': 1.0,
 'estudant

In [15]:
#Mostrando o resultado da função objetivo
obj = m.getObjective()
print(f"O valor da função objetivo é {round(obj.getValue(),2)}")

O valor da função objetivo é 1745.0
